<a href="https://colab.research.google.com/github/RiyaThakore/Real-time-emotion-analysis/blob/master/Real_time_emotion_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 
from PIL import Image
from tqdm import tqdm
import os

def atoi(s):
    n = 0
    for i in s:
        n = n*10 + ord(i) - ord("0")
    return n

folders = ['test','train']
emotions = ['angry', 'disgust', 'fearful', 'happy', 'sad', 'surprise', 'neutral']
os.makedirs('data', exist_ok=True)
for file in folder:
    os.makedirs(os.path.join('data',file), exist_ok=True)
    for inner in emotions:
        os.makedirs(os.path.join('data', file, inner), exist_ok=True)

angry, angry_test = 0, 0
disgust, disgust_test = 0, 0
fearful, fearful_test = 0, 0
happy, happy_test = 0, 0
sad, sad_test = 0, 0
surprise, surprise_test = 0, 0
neutral, neutral_test = 0, 0

df = pd.read_csv('C:/Users/riyan/Downloads/64681_127167_bundle_archive/fer20131.csv')
matrix = np.zeros((48,48),dtype=np.uint8)

for i in tqdm(range(len(df))):
    txt = df['pixels'][i]
    words = txt.split()
    for j in range(2304):
        xind = j // 48
        yind = j % 48
        matrix[xind][yind] = atoi(words[j])

    img = Image.fromarray(matrix)

    if i < 27250:
        if df['emotion'][i] == 0:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/train/angry/im'+str(angry)+'.png')
            angry += 1
        elif df['emotion'][i] == 1:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/train/disgust/im'+str(disgust)+'.png')
            disgusted += 1
        elif df['emotion'][i] == 2:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/train/fearful/im'+str(fearful)+'.png')
            fearful += 1
        elif df['emotion'][i] == 3:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/train/happy/im'+str(happy)+'.png')
            happy += 1
        elif df['emotion'][i] == 4:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/train/sad/im'+str(sad)+'.png')
            sad += 1
        elif df['emotion'][i] == 5:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/train/surprise/im'+str(surprise)+'.png')
            surprise += 1
        elif df['emotion'][i] == 6:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/train/neutral/im'+str(neutral)+'.png')
            neutral += 1

    else:
        if df['emotion'][i] == 0:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/test/angry/im'+str(angry_test)+'.png')
            angry_test += 1
        elif df['emotion'][i] == 1:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/test/disgust/im'+str(disgust_test)+'.png')
            disgusted_test += 1
        elif df['emotion'][i] == 2:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/test/fearful/im'+str(fearful_test)+'.png')
            fearful_test += 1
        elif df['emotion'][i] == 3:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/test/happy/im'+str(happy_test)+'.png')
            happy_test += 1
        elif df['emotion'][i] == 4:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/test/sad/im'+str(sad_test)+'.png')
            sad_test += 1
        elif df['emotion'][i] == 5:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/test/surprise/im'+str(surprise_test)+'.png')
            surprise_test += 1
        elif df['emotion'][i] == 6:
            img.save('C:/Users/riyan/Downloads/64681_127167_bundle_archive/test/neutral/im'+str(neutral_test)+'.png')
            neutral_test += 1

In [ ]:
import numpy as np
import argparse
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

train_dir = 'C:/Users/riyan/Downloads/64681_127167_bundle_archive/train'
test_dir = 'C:/Users/riyan/Downloads/64681_127167_bundle_archive/test'

num_train = 27250
num_test = 6784
batch_size = 64
num_epoch = 10

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer=Adam,metrics=['acc'])
model_info = model.fit_generator(train_generator,steps_per_epoch=num_train//batch_size,epochs=num_epoch,test_data=test_generator,test_steps=num_test//batch_size)
plot_model_history(model_info)
model.save_weights('model.h5')

model.load_weights('model.h5')

cv2.ocl.setUseOpenCL(False)
emotions_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #load in grayscale mode
    faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)
    #faces = cv2.CascadeClassifier.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

    for (x, y, width, height) in faces:
        cv2.rectangle(frame, (x, y), (x+width, y+height), (255, 0, 0), 2)
        roi_gray = gray[y:y + height, x:x + width]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(1000,1000),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1):
        break

cap.release()
cv2.destroyAllWindows()